# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [159]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [160]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)
        
print(diy_lu(a))

np.linalg.matrix_rank(a)

(array([[1.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 1.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 1.455, 1.   , 0.   , 0.   , 0.   ],
       [1.   , 1.714, 1.742, 1.   , 0.   , 0.   ],
       [1.   , 1.882, 2.276, 2.039, 1.   , 0.   ],
       [1.   , 2.   , 2.671, 2.944, 2.354, 1.   ]]), array([[ 3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,  3.000e+00,
         3.000e+00],
       [ 0.000e+00, -1.125e+00, -1.636e+00, -1.929e+00, -2.118e+00,
        -2.250e+00],
       [ 0.000e+00,  0.000e+00,  2.625e-01,  4.574e-01,  5.975e-01,
         7.013e-01],
       [ 0.000e+00,  2.220e-16,  0.000e+00, -2.197e-02, -4.480e-02,
        -6.469e-02],
       [ 0.000e+00, -4.528e-16,  0.000e+00,  6.939e-18,  8.080e-04,
         1.902e-03],
       [ 0.000e+00,  4.123e-16,  0.000e+00, -1.634e-17,  0.000e+00,
        -1.585e-05]]))


6

In [161]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [162]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [163]:

print(a)
a1 = a.copy()
a1[1, 1] = 3

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


Resulting matix still has full rank, but the naive LU routine breaks down.

In [164]:
np.linalg.matrix_rank(a1)

6

In [165]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/Users/ananasokeanov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/Users/ananasokeanov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/Users/ananasokeanov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/Users/ananasokeanov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [166]:

def minor_check(mat):
    i = 1
    while True and i != N + 1:
        mm = mat[:i,:i].copy()
        if np.linalg.det(mm) == 0:
            i += 1
            return False
            
        else:
            return True
            

print(minor(a), minor(a1))



True False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [173]:

def diy_lu_new(a):
   

    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    
    P = np.eye(a.shape[0])
    Pf = np.eye(a.shape[0])
    Ps = np.eye(a.shape[0])
    
    for i in range (N-1):
        
            m = np.argmax(a[i+1:, i])
            Pi = np.eye(a.shape[0])
            
            #if u[m+i+1, i] != u[i, i]:  
            s1 = Pi[m+i+1].copy()
            s2 = Pi[i].copy()
            Pi[m+i+1] = s2
            Pi[i] = s1
            P = P @ Pi
                

            u = Pi @ u

            lam = np.eye(N)
            gamma = u[i+1:, i] / u[i, i]

            lam[i+1:, i] = -gamma
            u = lam  @ u
            
            
            lam[i+1:, i] = gamma
            L = L @ Pi @ lam 

            
    
    
    return P.T @ L, u


print('разложение а:', '\n', diy_lu_new(a))


print('разложение а1:', '\n', diy_lu_new(a1))


разложение а: 
 (array([[ 1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 1.000e+00,  1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 1.000e+00,  1.571e+00,  1.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 1.000e+00,  1.941e+00,  1.959e+00,  1.000e+00,  0.000e+00,
         0.000e+00],
       [ 1.000e+00,  2.200e+00,  2.760e+00,  2.310e+00,  1.000e+00,
         0.000e+00],
       [ 1.000e+00, -2.200e+00,  1.288e+01, -1.617e+02,  3.640e+03,
         1.000e+00]]), array([[ 3.000e+00,  1.875e+00,  1.364e+00,  1.071e+00,  8.824e-01,
         7.500e-01],
       [ 0.000e+00, -5.114e-01, -4.813e-01, -4.193e-01, -3.651e-01,
        -3.214e-01],
       [ 0.000e+00,  0.000e+00,  4.484e-02,  5.615e-02,  5.725e-02,
         5.510e-02],
       [ 0.000e+00,  0.000e+00,  0.000e+00, -1.750e-03, -2.761e-03,
        -3.250e-03],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  3.589e-05,
         6.867e-05],
  

In [183]:
def diy_lu_newset(a):
    
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    
   
    Pf = np.eye(a.shape[0])
    Ps = np.eye(a.shape[0])
    
    for i in range (N-1):
        
            m = np.argmax(a[i+1:, i])
            Pi = np.eye(a.shape[0])
            
              
            s1 = Pi[m+i+1].copy()
            s2 = Pi[i].copy()
            Pi[m+i+1] = s2
            Pi[i] = s1
            

            u = Pi @ u

            lam = np.eye(N)
            gamma = u[i+1:, i] / u[i, i]

            lam[i+1:, i] = -gamma
            u = lam  @ u
            
            
            lam[i+1:, i] = gamma
            L = L @ Pi @ lam 

            A = Pi @ L @ u 
        
        
    return A, A-a

print(diy_lu_newset(a))

print(diy_lu_newset(a1))

(array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 1.875, 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.187],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231]]), array([[ 0.000e+00,  0.000e+00, -4.441e-16,  0.000e+00, -4.441e-16,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00, -5.551e-17,  1.110e-16,
         0.000e+00],
       [ 0.000e+00, -1.324e-01, -8.867e-02, -6.585e-02, -5.225e-02,
        -4.327e-02],
       [ 0.000e+00,  1.324e-01,  8.867e-02,  6.585e-02,  5.225e-02,
         4.327e-02]]))
(array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 3.   , 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
 

In [ ]:
#WIN
